In this notebook, I develop code to asses connectivity metrics across EEG nodes

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm, patches
import matplotlib.gridspec as gridspec
from scipy import signal
import umap
import mne
from mne.preprocessing import compute_current_source_density
from mne.connectivity import spectral_connectivity
from mne import EpochsArray

from tbd_eeg.data_analysis.eegutils import *
from tbd_eeg.data_analysis.Utilities import utilities as utils

from ipympl.backend_nbagg import Canvas
Canvas.header_visible.default_value = False
%matplotlib widget

In [2]:
epoch_cms = {
    'pre' : cm.Reds,
    'iso_high' : cm.PuOr,
    'iso_low' : cm.PuOr_r,
    'early_recovery': cm.Blues,
    'late_recovery' : cm.Greens
}

In [3]:
data_folder = "../tiny-blue-dot/zap-n-zip/eeg_pilot/mouse507190/pilot1_2020-02-28_10-33-11/recording1/"

# set the sample_rate for all data analysis
sample_rate = 2500

# load experiment metadata and eeg data
exp = EEGexp(data_folder)
eegdata = exp.load_eegdata(frequency=sample_rate, return_type='pd')

# locate valid channels (some channels can be disconnected and we want to ignore them in the analysis)
print('Identifying valid channels...')
median_amplitude = eegdata[:sample_rate*300].apply(
    utils.median_amplitude, raw=True, axis=0, distance=sample_rate
)
valid_channels = median_amplitude.index[median_amplitude < 2000].values
print('The following channels seem to be correctly connected and report valid data:')
print(list(valid_channels))

# load other data (running, iso etc)
print('Loading other data...')
running_speed = exp.load_running(return_type='pd')
iso = exp.load_analog_iso(return_type='pd')

# automatically annotate anesthesia epochs
iso_first_on = (iso>4).idxmax()
print('iso on at', iso_first_on)
iso_first_mid = ((iso[iso.index>iso_first_on]>1)&(iso[iso.index>iso_first_on]<4)).idxmax()
print('iso reduced at', iso_first_mid)
iso_first_off = (iso>1)[::-1].idxmax()
print('iso off at', iso_first_off)

# annotate artifacts with power in high frequencies
print('Annotating artifacts...')
hf_annots = pd.Series(
    eegdata[valid_channels].apply(
        find_hf_annotations, axis=0,
        sample_rate=sample_rate, fmin=300, pmin=0.25
    ).mean(axis=1),
    name='artifact'
)
recovery_first_jump = (hf_annots>4)[hf_annots.index>iso_first_off].idxmax()

epochs = pd.Series(
    index = [0, iso_first_on-0.001, iso_first_on+0.001, iso_first_mid-0.001,
             iso_first_mid+0.001, iso_first_off-0.001, iso_first_off+0.001,
             recovery_first_jump-0.001, recovery_first_jump+0.001, eegdata.index[-1]],
    data=['pre', 'pre', 'iso_high', 'iso_high', 'iso_low', 'iso_low',
          'early_recovery', 'early_recovery', 'late_recovery', 'late_recovery'],
    dtype=pd.CategoricalDtype(
        categories=['pre', 'iso_high', 'iso_low', 'early_recovery', 'late_recovery'],
        ordered=True
    )
)

No *stim* files were found. Experiment type: spontaneous.
Identifying valid channels...
The following channels seem to be correctly connected and report valid data:
[0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
Loading other data...


/home/saurabh.gandhi/Projects/.env_zz/lib/python3.7/site-packages/allensdk/brain_observatory/sync_dataset.py:454: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  return eval(self.dfile['analog_meta'].value)


iso on at 644.11
iso reduced at 1190.05
iso off at 1867.56
Annotating artifacts...


In [4]:
# validate data and artifact annotation
f, ax = plt.subplots(1, 1, figsize=(12, 2), tight_layout=True)
eegdata[[13, 4]][::20].plot(ax=ax, alpha=0.7)
ax2 = ax.twinx()
hf_annots.plot(ax=ax2, c='r', lw=0.5)
ax2.set_ylim(-np.sum(ax2.get_ylim())/2, ax2.get_ylim()[1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-63.55172413793103, 133.45862068965516)

# Connectivity Analysis (preprocessing)

## Reorder electrodes into proximal groups

In [5]:
egroups = {
    'left_front' : [11, 12, 13, 14],
    'right_front' : [18, 17, 16, 15],
    'left_front_middle' : [9, 10],
    'right_front_middle' : [20, 19],
    'left_back_middle' : [3, 4, 6, 7],
    'right_back_middle' : [26, 25, 23, 22],
    'left_back_middle_center' : [5, 8],
    'right_back_middle_center' : [24, 21],
    'left_back' : [1, 2],
    'right_back' : [28, 27],
    'left_bottom' : [0],
    'right_bottom' : [29],
}

exp.ch_coordinates['z'] = 0
exp.ch_coordinates['group'] = ''
exp.ch_coordinates['gid'] = 0
exp.ch_coordinates['wgid'] = 0
for i, (g, idx) in enumerate(egroups.items()):
    exp.ch_coordinates.loc[idx, 'group'] = g
    exp.ch_coordinates.loc[idx, 'gid'] = i
    exp.ch_coordinates.loc[sorted(idx), 'wgid'] = idx
exp.ch_coordinates = exp.ch_coordinates.sort_values(['gid', 'wgid'])
exp.ch_coordinates['order'] = 0
_left = exp.ch_coordinates.index[exp.ch_coordinates.group.str.contains('left')]
exp.ch_coordinates.loc[_left, 'order'] = range(len(_left))
_right = exp.ch_coordinates.index[exp.ch_coordinates.group.str.contains('right')]
exp.ch_coordinates.loc[_right, 'order'] = len(_left)+np.arange(len(_right))[::-1]
exp.ch_coordinates.sort_index(inplace=True)
exp.ch_coordinates.drop('wgid', inplace=True, axis=1)

exp.ch_coordinates

,AP,ML,z,group,gid,order
0,-4.14,-4.05,0,left_bottom,10,14
1,-4.14,-2.24,0,left_back,8,12
2,-4.14,-1.00,0,left_back,8,13
3,-3.04,-4.13,0,left_back_middle,4,6
4,-3.04,-2.88,0,left_back_middle,4,7
5,-3.04,-1.13,0,left_back_middle_center,6,10
6,-1.96,-4.05,0,left_back_middle,4,8
7,-1.96,-2.88,0,left_back_middle,4,9
8,-1.96,-1.12,0,left_back_middle_center,6,11
9,-0.48,-3.50,0,left_front_middle,2,4


In [6]:
# define a function to quickly plot the electrode map with or without borders
def plot_electrode_map(ax, highlight=None, labels=True, cmap=cm.Paired, s=50):
    colors = np.array(exp.ch_coordinates.gid.map(lambda x: cmap(x/11, 0.9)))
    if highlight in set(exp.ch_coordinates.group):
        colors = np.array(exp.ch_coordinates.apply(lambda row: cmap(row.gid/12, 0.9) if row.group==highlight else cm.Greys(0.5,0.5), axis=1))
    exp.ch_coordinates.plot(
        kind='scatter', x='ML', y='AP', marker='o', ax=ax, legend=False, c=colors, s=s
    )
    if labels:
        for i in exp.ch_coordinates.index:
            ax.annotate(exp.ch_coordinates.loc[i, 'order'], exp.ch_coordinates.loc[i, ['AP', 'ML']][::-1]+[0, 0.2], xycoords='data', ha='center')
    ax.set_xlim(-5, 5)
    ax.set_ylim(-5, 3)
    if labels:
        ax.set_title('Electrode map')
    if not labels:
        ax.set_xlabel('')
        ax.set_ylabel('')
    ax.set_xticks([])
    ax.set_yticks([]);

# function to show electrode groups along an axis instead of electrode numbers
def draw_groups(ax, cmap=cm.Paired):
    ax.set_xlim(-0.5, 29.5)
    ax.set_ylim(29.5, -0.5)
    ax.set_xticks([])
    ax.set_yticks([])
    nt = 0
    def add_patch(df):
        nonlocal nt, ax
        ax.add_patch(patches.Rectangle(
            (nt+0.05-0.5, 30), len(df)-0.05, 1, clip_on=False, color=cmap(df.gid.iloc[0]/12, 0.9), label=df.group.iloc[0]
        ))
        ax.add_patch(patches.Rectangle(
            (-2, nt+0.05-0.5), 1, len(df), clip_on=False, color=cmap(df.gid.iloc[0]/12, 0.9)
        ))
        nt += len(df)
    exp.ch_coordinates.sort_values('order').groupby('gid', sort=False).apply(add_patch)

In [7]:
f, ax = plt.subplots(1, 1, figsize=(4, 4), constrained_layout=False)
plot_electrode_map(ax, labels=True, s=50)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

---

## Generate valid windows
Valid meaning windows without artifacts.  
Turns out, we can just generate windows very easily, and validate them on the fly using a validity Series, `invalid_times`.

In [8]:
thresh = 4
invalid_times = (hf_annots>thresh)
invalid_times[invalid_times]

31.052911      True
33.897311      True
48.119311      True
65.623311      True
66.498511      True
               ... 
3606.244911    True
3615.215711    True
3617.622511    True
3618.935311    True
3630.531711    True
Name: artifact, Length: 730, dtype: bool

In [9]:
def drop_artifacts(df, winsize, sample_rate):
    if (len(df) < winsize*sample_rate) | (df['artifact'].sum() > 0):
        return pd.Series([np.nan]*(int(winsize*sample_rate)))
    return pd.Series(df.index[:int(winsize*sample_rate)])

In [10]:
WINSIZE_s = 1 # lets look at 5 s long states
windows = pd.Series(index=eegdata.index, data=(eegdata.index/WINSIZE_s).astype(int), name='window')
aligned_windows = pd.concat([windows, invalid_times.reindex(windows.index, method='nearest')], axis=1)
valid_windows = aligned_windows.groupby('window').apply(drop_artifacts, winsize=WINSIZE_s, sample_rate=sample_rate).dropna()
win_center = valid_windows.apply(lambda x: x.mean(), axis=1)
win_epoch = pd.Series(data=epochs.reindex(win_center, method='nearest').values, index=win_center.index)
valid_windows['epoch'] = win_epoch

In [11]:
valid_wins_by_epoch = valid_windows.set_index('epoch', append=True).swaplevel()
print('Number of workable windows in each epoch:')
[print(idx, len(valid_wins_by_epoch.loc[idx])) for idx in list(valid_wins_by_epoch.index.levels[0])];

Number of workable windows in each epoch:
pre 402
iso_high 501
iso_low 677
early_recovery 261
late_recovery 1290


# Coherence
Some things to keep in mind:
1. Coherence is susciptible to volume conduction. A good way to get around this is to use only the imaginary part of coherence (volume conduction signals should have a nearly zero delay (speed of light), and thus, in the cross-spectrum, will have zero phase lag, and contribute only to the real part of coherence).
1. Coherence depends on taking the fourier transform. The longer the signal, the more stable the estimation of fourier transform. However, it also assumes stationarity of the signal, i.e. the fourier components do not change much in time within the window. A shorter signal guarantees stationarity better. Thus there is a trade off and it needs to be deicded what is the best window.  
   For now, I will use a 5s long window. This is 5x the minimum frequency of 1 Hz that we care about, so long enough but not super long. For higher frequency bands, this window might be too long?

In [ ]:
from mne.connectivity import spectral_connectivity
from mne import EpochsArray

connectivity, freq, t, nep, nta = spectral_connectivity(
    data, method='coh', mode='fourier', sfreq=sample_rate, faverage=False, fmax=100, block_size=2, n_jobs=8
)

# data : array[#trials x #channels x time]
# connectivity : array[#channels x #channels x #freqs]
# freq : array[#freqs] # frequencies are evenly sampled up to fmax

In [19]:
_data = [eegdata.loc[valid_wins_by_epoch.loc[ep].loc[x]].values.T[exp.ch_coordinates.order] for x in valid_wins_by_epoch.loc[ep].index]

In [26]:
connectivity[ep].shape

(30, 30, 96)

In [21]:
_data[0].shape

(30, 2500)

In [12]:
connectivity = {}
for ep in valid_wins_by_epoch.index.levels[0]:
    connectivity[ep], freq, t, nep, nta = spectral_connectivity(
        [eegdata.loc[valid_wins_by_epoch.loc[ep].loc[x]].values.T[exp.ch_coordinates.order] for x in valid_wins_by_epoch.loc[ep].index],
        method='pli', mode='fourier', sfreq=sample_rate, faverage=False, fmax=100, block_size=2, n_jobs=8
    )

Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 435 connections
    using t=0.000s..1.000s for estimation (2500 points)
    frequencies: 5.0Hz..100.0Hz (96 points)
    using FFT with a Hanning window to estimate spectra
    the following metrics will be computed: PLI
    computing connectivity for epochs 1..8


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:   14.5s remaining:   24.2s


    computing connectivity for epochs 9..16
    computing connectivity for epochs 17..24
    computing connectivity for epochs 25..32
    computing connectivity for epochs 33..40
    computing connectivity for epochs 41..48
    computing connectivity for epochs 49..56


[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:   15.2s remaining:    9.1s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:   15.2s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.0s remaining:    0.0s
[Para

    computing connectivity for epochs 57..64
    computing connectivity for epochs 65..72
    computing connectivity for epochs 73..80
    computing connectivity for epochs 81..88
    computing connectivity for epochs 89..96
    computing connectivity for epochs 97..104
    computing connectivity for epochs 105..112


...

[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.0s
[Para

    computing connectivity for epochs 1193..1200
    computing connectivity for epochs 1201..1208
    computing connectivity for epochs 1209..1216
    computing connectivity for epochs 1217..1224
    computing connectivity for epochs 1225..1232
    computing connectivity for epochs 1233..1240
    computing connectivity for epochs 1241..1248


[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.0s
[Para

    computing connectivity for epochs 1249..1256
    computing connectivity for epochs 1257..1264
    computing connectivity for epochs 1265..1272
    computing connectivity for epochs 1273..1280
    computing connectivity for epochs 1281..1288
    computing connectivity for epochs 1289..1290
    assembling connectivity matrix (filling the upper triangular region of the matrix)
[Connectivity computation done]


[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.0s
[Para

---
connectivity metric as a function of distance between electrodes

In [13]:
distances = np.zeros((30, 30))
df = exp.ch_coordinates.copy()
for i in df.index:
    for j in df.index:
        distances[i, j] = np.sqrt((df.loc[i, 'AP']-df.loc[j, 'AP'])**2 + (df.loc[i, 'ML']-df.loc[j, 'ML'])**2)

mask = connectivity['pre'][:, :, 0] > 0
distances = distances[mask]
sorted_idx = distances.argsort()
distances = distances[sorted_idx]
metric = connectivity['pre'][mask, :][sorted_idx, :]

In [14]:
freq_idx = 2
f, ax = plt.subplots(1, 1, figsize=(4, 3), tight_layout=True)
m, c = np.polyfit(distances, metric[:, freq_idx], 1)
corr = np.corrcoef(distances, metric[:, freq_idx])[0, 1]
ax.scatter(distances, metric[:, freq_idx], s=3)
ax.plot(distances, m*distances+c, c='r', label='R^2 = {0:.3f}'.format(corr**2))
ax.legend(loc=1, fontsize=10);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

---

In [15]:
# split into appropriate bands
# 0-10, 10-20, 20-30, 30-40, 40-60, 60-80
bands = [(0, 10), (10, 20), (20, 30), (30, 40), (40, 60), (60, 80)]
band_connectivity = {}
for ep in valid_wins_by_epoch.index.levels[0]:
    band_connectivity[ep] = {}
    for band in bands:
        if sum((freq<band[1])&(freq>band[0])) > 0: # there are some frequencies within the band
            connectivity_rs = connectivity[ep][:, :, (freq<band[1])&(freq>band[0])]
            band_connectivity[ep][band] = np.abs(connectivity_rs.mean(axis=-1))

* If common reference were driving high coherence, it should not go away during anesthesia. However, coherence does decrease to almost zero during anesthesia in some frequency bands, suggesting that common reference is not a problem.

In [16]:
f, ax = plt.subplots(1, 1, figsize=(4, 3))
ax.hist(band_connectivity['pre'][(10, 20)].flatten(), log=True);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [42]:
# connectivity difference (pre - iso_low) in different frequency bands
f, axes = plt.subplots(
    1, len(band_connectivity['pre']),
    figsize=(2.4*len(band_connectivity), 2.4),
    constrained_layout=True, sharex=True, sharey=True
)
for r, (b, ax) in enumerate(zip(band_connectivity['pre'].keys(), axes)):
    im = ax.imshow(
        (band_connectivity['pre'][b][:, :]-band_connectivity['iso_low'][b][:, :]),#/(band_connectivity['pre'][b][:, :]+band_connectivity['iso_low'][b][:, :]+1e-5),
        cmap=cm.PuOr_r, vmin=-0.5, vmax=0.5
    )
    ax.axvline(band_connectivity[ep][b].shape[0]/2-0.5, 0, 0.5, c='k')
    ax.axhline(band_connectivity[ep][b].shape[1]/2-0.5, 0, 0.5, c='k')
    draw_groups(ax)
    ax.set_title(b)
plt.colorbar(im, ax=ax, aspect=50, shrink=0.75);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [43]:
f, ax = plt.subplots(1, 1, figsize=(5, 4), tight_layout=True)
im = ax.imshow(
    band_connectivity['pre'][(0, 10)][:, :],
    cmap=cm.PuOr_r, vmin=0, vmax=0.3
)
ax.axvline(band_connectivity[ep][b].shape[0]/2-0.5, 0, 0.5, c='k')
ax.axhline(band_connectivity[ep][b].shape[1]/2-0.5, 0, 0.5, c='k')
draw_groups(ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
f, axes = plt.subplots(
    len(band_connectivity['pre']),
    len(band_connectivity),
    figsize=(
        2*len(band_connectivity),
        2*len(band_connectivity['pre']),
    ), tight_layout=True, sharex=True, sharey=True
)
for c, (ep, axe) in enumerate(zip(band_connectivity.keys(), axes.T)):
    for r, (b, ax) in enumerate(zip(band_connectivity[ep].keys(), axe)):
        im = ax.imshow(
            band_connectivity[ep][b][:, :],
            cmap=cm.PuOr_r, vmin=0, vmax=1
        )
        ax.axvline(band_connectivity[ep][b].shape[0]/2-0.5, 0, 0.5, c='k')
        ax.axhline(band_connectivity[ep][b].shape[1]/2-0.5, 0, 0.5, c='k')
        draw_groups(ax)
        if r == 0:
            ax.set_title(ep)
        if c == 0:
            ax.set_ylabel('{0:d}-{1:d} Hz'.format(*b))
axes.flatten()[0].legend(loc=(0, 1.15), ncol=4, fontsize=8)
plt.subplots_adjust(left=0.02, right=0.99, top=0.92, bottom=0.02);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
averaging_win = 2*int(1/np.diff(freq).mean())
n_groups = int(len(set(exp.ch_coordinates.group)) / 2) + 1
axes = []
f = plt.figure(figsize=(1.5*n_groups, 1.5*n_groups), constrained_layout=True)
sp = gridspec.GridSpec(ncols=n_groups, nrows=n_groups, figure=f, width_ratios=[0.5]+[1]*(n_groups-1), height_ratios=[0.5]+[1]*(n_groups-1))
for i, name_1 in enumerate(exp.ch_coordinates.groupby('gid').apply(lambda df: df.group.iloc[0] if df.gid.iloc[0]%2 == 0 else np.nan).dropna()):
    indices_1 = exp.ch_coordinates[exp.ch_coordinates.group==name_1].order.values[:, np.newaxis]
    for j, name_2 in enumerate(exp.ch_coordinates.groupby('gid').apply(lambda df: df.group.iloc[0] if df.gid.iloc[0]%2 == 0 else np.nan).dropna()):
        indices_2 = exp.ch_coordinates[exp.ch_coordinates.group==name_2].order.values[np.newaxis, :]
        for epoch in ['pre', 'iso_low']:
            con = connectivity[epoch][indices_1, indices_2, :].mean(axis=0).mean(axis=0)
            con = np.convolve(con, np.ones(averaging_win)/averaging_win, mode='same')
            if len(axes) == 0:
                ax = None
            else:
                ax = axes[0]
            axes.append(f.add_subplot(sp[i+1, j+1], sharex=ax, sharey=ax, label=f'{name_1}:{name_2}'))
            axes[-1].plot(freq, con, label=f'{epoch}', alpha=0.8)
            xticks = axes[-1].get_xticks()
            axes[-1].set_xticks([])
            axes[-1].set_yticks([])
        if j == 0:
            axes[-1].set_yticks([0, 0.5, 1])
        if i == n_groups-2:
            axes[-1].set_xticks([0, 50, 99])
axes[0].set_xlim(0, 99)
for i, name_1 in enumerate(exp.ch_coordinates.groupby('gid').apply(lambda df: df.group.iloc[0] if df.gid.iloc[0]%2 == 0 else np.nan).dropna()):
    ax = f.add_subplot(sp[0, i+1], label='x_{0:d}'.format(i), aspect='equal')
    plot_electrode_map(ax, labels=False, highlight=name_1, s=20, cmap=cm.Dark2)
    ax = f.add_subplot(sp[i+1, 0], label='x_{0:d}'.format(i), aspect='equal')
    plot_electrode_map(ax, labels=False, highlight=name_1, s=20, cmap=cm.Dark2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/saurabh.gandhi/Projects/.env_zz/lib/python3.7/site-packages/ipykernel_launcher.py:17: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.


# SCoT

In [ ]:
import scot
import scot.backend_sklearn

In [76]:
data, trial = [], []
for (t, w) in valid_wins_by_epoch.index:
    if t in ['pre', 'iso_low']:
        trial.append(t)
        data.append(eegdata.loc[valid_wins_by_epoch.loc[t].loc[w], valid_channels].values)
data = np.array(data).transpose(0, 2, 1)
data.shape

(63, 30, 10000)

In [93]:
exp.ch_coordinates['z'] = 0
ws = scot.Workspace({'model_order': 200}, reducedim=6, fs=sample_rate, locations=exp.ch_coordinates.values)

## Run steps individually to understand how SCoT works

## Combined running steps

In [94]:
ws.set_data(data, trial)
ws.do_mvarica()

#p = ws.var_.test_whiteness(50)
# print('Whiteness:', p)

/home/saurabh.gandhi/Projects/.env_zz/lib/python3.7/site-packages/scot/external/infomax_.py:183: RuntimeWarning: overflow encountered in exp
  y = 1.0 / (1.0 + np.exp(-u))


scot.varica.mvarica.<locals>.Result

In [95]:
f = plt.figure(figsize=(10, 8), tight_layout=True)

# Configure plotting options
ws.plot_f_range = [0, 100]  # only show 0-30 Hz
ws.plot_diagonal = 'S'  # put spectral density plots on the diagonal
ws.plot_outside_topo = True  # plot topos above and to the left

ws.set_used_labels(['pre'])
ws.fit_var()
ws.plot_connectivity_topos(f)
c, _ = ws.get_connectivity('COH', f)

ws.set_used_labels(['iso_low'])
ws.fit_var()
# ws.plot_connectivity_topos(f)
c, _ = ws.get_connectivity('COH', f)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/saurabh.gandhi/Projects/.env_zz/lib/python3.7/site-packages/scot/eegtopo/projections.py:46: RuntimeWarning: invalid value encountered in arcsin
  alphas[mask] = arcsin(betas[mask]) / betas[mask]
/home/saurabh.gandhi/Projects/.env_zz/lib/python3.7/site-packages/scot/plotting.py:264: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  ax = fig.add_subplot(m+1, m+1, j + (i+1) * (m+1) + 2)
